In [2]:
using Pkg
Pkg.activate("..")

 Activating environment at `~/Nextcloud/Work/phd_uppsala/projects/nonlinear system identification/Twisted Particle Filter/Project.toml`


In [3]:
using LowLevelParticleFilters
using LinearAlgebra
using GaussianSmoothers
using Distributions
using StaticArrays

In [12]:
n = 4   # Dimension of state
m = 2   # Dimension of input
p = 2   # Dimension of measurements
N = 500 # Number of particles
T = 1000

const dg = MvNormal(p,1.0)          # Measurement noise Distribution
const df = MvNormal(n,1.0)          # Dynamics noise Distribution
const d0 = MvNormal(randn(n),2.0)   # Initial state Distribution

Tr = randn(n,n)
const A = SMatrix{n,n}(Tr*diagm(0=>LinRange(0.5,0.95,n))/Tr)
const B = @SMatrix randn(n,m)
const C = @SMatrix randn(p,n)

dynamics(x,u) = A*x .+ B*u
measurement(x) = C*x
vecvec_to_mat(x) = copy(reduce(hcat, x)') # Helper function

pf = ParticleFilter(N, dynamics, measurement, df, dg, d0)
xs,u,y = LowLevelParticleFilters.simulate(pf,T,df) # We can simulate the model that the pf represents

eye(n) = Matrix{Float64}(I,n,n)
kf     = LowLevelParticleFilters.KalmanFilter(A, B, C, 0, eye(n), eye(p), MvNormal([1.,1.]))
xf,xt,R,Rt,ll = forward_trajectory(kf, u, y) # filtered, prediction, pred cov, filter cov, loglik
xT,R,lls = smooth(kf, u, y) # Smoothed state, smoothed cov, loglik

LoadError: invalid redefinition of constant A

In [9]:
LinearDynamicsModel(A, B, eye(p))

LoadError: MethodError: no method matching LinearDynamicsModel(::SArray{Tuple{2,2},Float64,2,4}, ::SArray{Tuple{2,2},Float64,2,4}, ::Array{Float64,2})
Closest candidates are:
  LinearDynamicsModel(::T, ::T) where T at /home/johan/Nextcloud/Work/phd_uppsala/projects/nonlinear system identification/Twisted Particle Filter/src/GaussianSmoothers/src/models.jl:60
  LinearDynamicsModel(::T, ::T, !Matched::T) where T at /home/johan/Nextcloud/Work/phd_uppsala/projects/nonlinear system identification/Twisted Particle Filter/src/GaussianSmoothers/src/models.jl:56
  LinearDynamicsModel(::T, ::T, !Matched::T, !Matched::T) where T at /home/johan/Nextcloud/Work/phd_uppsala/projects/nonlinear system identification/Twisted Particle Filter/src/GaussianSmoothers/src/models.jl:47

In [11]:
@time forward_trajectory(kf, u, y);

  0.001151 seconds (2.91 k allocations: 291.359 KiB)
